In [ ]:
#Importando bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
#Nomes para as colunas
columns = ["Estacao","Data","Hora","Prec","TempMax","TempMin","Insolacao","EvapoPiche","TempCompMedia",
           "UmidRelatMedia", "VelocVento", "Missing"]

#Tipos de dados em cada coluna
types = {"Hora":np.uint16,"Prec":np.float64,"TempMax":np.float64,"TempMin":np.float64,"Insolacao":np.float64,
         "EvapoPiche":np.float64,"TempCompMedia":np.float64,"UmidRelatMedia":np.float64,"VelocVento":np.float64}

#Leitura do arquivo csv
dataSet = pd.read_csv("Dados_INMET/Diarios/INMET-Dados_Diarios_Natal_1968-2018.csv",sep=';',skiprows=48, nrows = 27003,
          usecols = columns[1:11], names=columns, dtype = types)

In [ ]:
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [ ]:
#Verificando dados
dataSet.head()

In [ ]:
#Verificando o tipo dos dados
dataSet.dtypes

In [ ]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]

In [ ]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSet00.head()

In [ ]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSet12.head()

In [ ]:
#Transformando a data no índice de cada dataframe
dataSet00 = dataSet00.set_index("Data")
dataSet12 = dataSet12.set_index("Data")

In [ ]:
#Descartando a coluna "Hora"
dataSet00, dataSet12 = dataSet00.drop(columns = "Hora"), dataSet12.drop(columns = "Hora")

In [ ]:
#Juntando os dados em um mesmo dataframe (no caso, o dataSet00)
for i in dataSet00.index:
    try:
        dataSet00["Prec"].loc[i] = dataSet12["Prec"].loc[i]
        dataSet00["TempMin"].loc[i] = dataSet12["TempMin"].loc[i]
    except:
        print("Data " + str(i.day) + "/" + str(i.month) + "/" + str(i.year) + " Não encontrada!")

In [ ]:
#Atribuindo à variável dataSet o DataFrame atualizado
dataSet = dataSet00
#Verificando dimensões do DataFrame
print(dataSet.shape)

In [ ]:
#Eliminando dias que estejam faltando variáveis
dataSet = dataSet.dropna()

In [ ]:
#Verificando dimensões do DataFrame após excluir linhas com problema
print(dataSet.shape)

In [ ]:
#Transformando a coluna Data de volta em uma coluna de informações do dataSet
dataSet = dataSet.reset_index()

In [ ]:
#Transformando cada elemento da label Data em três novas labels: dia, mes e ano
mes, ano,  = [], []
for i in dataSet.index:
    mes.insert(i, dataSet.loc[i, "Data"].month)
    ano.insert(i, dataSet.loc[i, "Data"].year)

In [ ]:
#Transformando cada elemento da label Data na quantidade de dias passados a contar da data 01/01/1968
data_referencia = datetime.strptime('1968-01-01', '%Y-%m-%d')
for i in dataSet.index:
    dataSet.loc[i, "Data"] = (dataSet.loc[i, "Data"] - data_referencia).days

In [ ]:
#Adicionando as labels para o DataFrame dataSet e eliminando a coluna "Data"
dataSet["Mes"], dataSet["Ano"] = mes, ano

In [ ]:
#Visualizando os dados
dataSet.head()

In [ ]:
#Separando a variável que será prevista (Y) em função das demais variáveis (X) e convertendo-os em um ndarray
X = np.array(dataSet.drop(columns = "TempMax"))
Y = np.array(dataSet["TempMax"])

In [ ]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 1)

In [ ]:
#Visualizando o conjunto de treino
print(X_train)
print(X_train.shape)

In [ ]:
#Efetuando uma transformação do tipo: z = (x - u) / s, normalizando em termos do desvio padrão
sc = StandardScaler()
X_train[:,:-2] = sc.fit_transform(X_train[:,:-2])
X_test[:,:-2] = sc.transform(X_test[:,:-2])

In [ ]:
#Verificando os dados de treino após a transformação
print(X_train)

In [ ]:
#Criando um modelo de MLP
mlp = Sequential()
#Adicionando a camada de entrada
mlp.add(Dense(activation = 'relu', input_dim = len(X_train[0]), units = 40, kernel_initializer = 'uniform'))
#Adicionando a segunda camada
mlp.add(Dense(activation = 'relu', units = 20, kernel_initializer = 'uniform'))
#Adicionando a terceira camada
mlp.add(Dense(activation = 'relu', units = 10, kernel_initializer = 'uniform'))
#Adicionando a quarta camada
mlp.add(Dense(activation = 'relu', units = 5, kernel_initializer = 'uniform'))
#Adicionando a camada de saída
mlp.add(Dense(units = 1, kernel_initializer='normal'))
# Compilando o modelo
mlp.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
#Mostrando um sumário do modelo de MLP criada
mlp.summary()

In [ ]:
#Treinando a MLP rede, a partir do modelo de MLP criado
rede = mlp.fit(X_train, Y_train, epochs=100, verbose=1, validation_split=0.1)

In [ ]:
#Mostrando o processo de treino graficamente
plt.plot(rede.history['loss'])
plt.plot(rede.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim(0,1)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Fazendo uma previsão com base nos dados de teste
Y_previsto = mlp.predict(X_test)

In [ ]:
#Mostrando o array de dados previstos
print(Y_previsto)

In [ ]:
#Mostrando o array de dados reais que deveriam ter sido previstos no passo anterior
print(Y_test)

In [ ]:
#Calculando e mostrando o erro mínimo quadrático entre o Y previsto e o Y real
print(mean_squared_error(Y_previsto, Y_test))